In [20]:
import torch
import torch.nn.functional as F
import torchvision.models as models
import matplotlib.pyplot as plt
import sys
import getopt
import pathlib
import shutil
import numpy as np
from matplotlib.image import imsave
from glob import glob
from os.path import join
from utils import *
from cam.scorecam import ScoreCAM
import warnings

In [21]:
## EXPERIMENT 1: Class discrimination

# clear results directory
warnings.filterwarnings("ignore")
path_results = pathlib.Path('./results')
if path_results.exists() and path_results.is_dir():
    shutil.rmtree(path_results) #rm previous results
path_results.mkdir(parents=True, exist_ok=True) 

# init
TARGET_LAYER = "features.29"
IMAGE_DIR = "images/X_ILSVRC2012"
MEANS = [0.485, 0.456, 0.406]
STDS = [0.229, 0.224, 0.225]
SIZE = 224

# load model
model = models.vgg16(pretrained=True).eval()
model_scorecam = ScoreCAM(dict(arch=model, layer_name=TARGET_LAYER, input_size=(SIZE, SIZE)))

# load images
images = glob(join(IMAGE_DIR, "*"))

# preprocess image + generate heatmaps
for index, image in enumerate(images):
    
    # preprocess
    input_image = load_image(image)  
    input_ = apply_transforms_score(input_image, MEANS, STDS, SIZE)
    if torch.cuda.is_available():
        input_ = input_.cuda()

    # get class
    # predicted_class = model(input_).max(1)[-1]
    
    # get heatmap
    heatmap = model_scorecam(input_)


Target layer:  ReLU(inplace=True)


In [22]:
# visualize heatmaps
input_ = input_.cpu() 
heatmap = heatmap.type(torch.FloatTensor).cpu()
imgs = [(input_image,heatmap,heatmap),(input_image,heatmap,heatmap)]
labels = [("yo", "ya", "ye"),("yo", "ya", "ye")]

with torch.no_grad():
  visualize_exp1(imgs,labels )





NameError: name 'visualize_exp1' is not defined